<a href="https://colab.research.google.com/github/aisancesg/Tinkoff_course_ADI/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install psycopg2-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.4 MB/s eta 0:00:00


In [ ]:
import psycopg2
import pandas as pd
import numpy  as np
from plotly import graph_objects as go
from pandas import DataFrame, read_csv
from requests import post
conn = psycopg2.connect( host="158.160.52.106", port=5432, database="postgres", user="student", password="JvLda93aA",
)
cur = conn.cursor()
cur.execute("SELECT * FROM msu_analytics.game")
data = cur.fetchall()
df_by_months=pd.read_sql("select t.months, count(distinct t.phone_num) as cnt_visitors, count (distinct t.password) as cnt_accounts, count (distinct application_rk) as cnt_applications, sum(game_flg) as cnt_games_played from (select *, DATE_TRUNC('MONTH', c.visit_dttm) as months, DATE_TRUNC('WEEK', c.visit_dttm) as weeks from msu_analytics.client c left join msu_analytics.account acc on c.client_rk = acc.client_rk left join msu_analytics.application app on acc.account_rk = app.account_rk left join msu_analytics.game g on app.game_rk = g.game_rk order by c.visit_dttm ) as t group by t.months order by t.months", conn)
df_by_months

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,months,cnt_visitors,cnt_accounts,cnt_applications,cnt_games_played
0,2022-09-01,778,190,91,59.0
1,2022-10-01,843,217,109,74.0
2,2022-11-01,786,200,101,71.0
3,2022-12-01,805,196,100,39.0
4,2023-01-01,852,213,111,21.0
5,2023-02-01,32,8,0,NaN


In [ ]:
df_by_weeks=pd.read_sql("select t.weeks, count(distinct t.phone_num) as cnt_visitors, count (distinct t.password) as cnt_accounts, count (distinct application_rk) as cnt_applications, sum(game_flg) as cnt_games_played from (select *, DATE_TRUNC('MONTH', c.visit_dttm) as months, DATE_TRUNC('WEEK', c.visit_dttm) as weeks from msu_analytics.client c left join msu_analytics.account acc on c.client_rk = acc.client_rk left join msu_analytics.application app on acc.account_rk = app.account_rk left join msu_analytics.game g on app.game_rk = g.game_rk order by c.visit_dttm ) as t group by t.weeks order by t.weeks", conn)
df_by_weeks

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,weeks,cnt_visitors,cnt_accounts,cnt_applications,cnt_games_played
0,2022-08-29,111,32,15,8
1,2022-09-05,184,31,9,6
2,2022-09-12,167,43,27,11
3,2022-09-19,181,45,21,18
4,2022-09-26,195,59,34,26
5,2022-10-03,204,58,27,22
6,2022-10-10,190,44,20,17
7,2022-10-17,171,41,20,14
8,2022-10-24,189,48,27,11
9,2022-10-31,197,47,31,23


In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=5,
                    subplot_titles=["Посетившие сайт в сентябре","Посетившие сайт в октябре", "Посетившие сайт в ноябре", "Посетившие сайт в декабре", "Посетившие сайт в январе"]
                    ,column_widths=[11, 11, 11, 11, 11])
for i in range(5):
  fig.add_trace(go.Funnel(
    y = ["Site visit", "Registration", "Application", "Game played"],
    x = [df_by_months.cnt_visitors[i], df_by_months.cnt_accounts[i] , df_by_months.cnt_applications[i], df_by_months.cnt_games_played[i]],
    textinfo = "value+percent initial"),
    row=1, col=i+1)

fig.update_layout(showlegend=False,height=500, width=1400)
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=6,
                    subplot_titles=[str(i)[:10] for i in df_by_weeks['weeks'].values])

for i in range(6):
  fig.add_trace(go.Funnel(
    y = ["Site visit", "Registration", "Application", "Game played"],
    x = [df_by_weeks.cnt_visitors[i], df_by_weeks.cnt_accounts[i] , df_by_weeks.cnt_applications[i], df_by_weeks.cnt_games_played[i]],
    textinfo = "value+percent initial"),
    row=1, col=i+1)
for i in range(6, 12):
  fig.add_trace(go.Funnel(
    y = ["Site visit", "Registration", "Application", "Game played"],
    x = [df_by_weeks.cnt_visitors[i], df_by_weeks.cnt_accounts[i] , df_by_weeks.cnt_applications[i], df_by_weeks.cnt_games_played[i]],
    textinfo = "value+percent initial"),
    row=2, col=i-5)
for i in range(12, 18):
  fig.add_trace(go.Funnel(
    y = ["Site visit", "Registration", "Application", "Game played"],
    x = [df_by_weeks.cnt_visitors[i], df_by_weeks.cnt_accounts[i] , df_by_weeks.cnt_applications[i], df_by_weeks.cnt_games_played[i]],
    textinfo = "value+percent initial"),
    row=3, col=i-11)
for i in range(18, 23):
  fig.add_trace(go.Funnel(
    y = ["Site visit", "Registration", "Application", "Game played"],
    x = [df_by_weeks.cnt_visitors[i], df_by_weeks.cnt_accounts[i] , df_by_weeks.cnt_applications[i], df_by_weeks.cnt_games_played[i]],
    textinfo = "value+percent initial"),
    row=4, col=i-17)

fig.update_layout(showlegend=False,height=2000, width=1400)
fig.show()

Выводы: В декабре просела доля пришедших на игру. Из конверсий по неделям видно, что потерявшиеся люди посещали сайт в середине-конца декабря. Возможно, в предновогодней суете они не оценили время. Возможно, стоит у них уточнять за несколько дней до игры, осталось ли всё в силе, чтоб при случае освободить слот времени для других.

Дополнительно я сделала запрос (см. ниже), исходя из которого можно понять, что у большинства из тех, кто не пришел на игру, промежуток между заявкой и датой игры большой, возможно они забывают об игре. Предполагаю, что стоит в таком случае присылать напоминание.


```
select *,  


  DATE_TRUNC('MONTH', c.visit_dttm) as months,
  
  DATE_TRUNC('WEEK', c.visit_dttm)  as weeks,
  (case

		when extract('doy'from acc.registration_dttm) - extract('doy' from c.visit_dttm)<0

		then extract('doy'from acc.registration_dttm) - extract('doy' from c.visit_dttm)+365

		else extract('doy'from acc.registration_dttm) - extract('doy' from c.visit_dttm)

	end) as days_between_visit_reg,

	(case
		when extract('doy'from app.application_dttm) - extract('doy' from acc.registration_dttm) <0

		then extract('doy'from app.application_dttm) - extract('doy' from acc.registration_dttm) +365

		else extract('doy'from app.application_dttm) - extract('doy' from acc.registration_dttm)

	end) as days_between_reg_app ,

	(case
		when extract('doy' from g.game_dttm) - extract('doy'from app.application_dttm) <0

		then extract('doy' from g.game_dttm)  - extract('doy'from app.application_dttm) +365

		else extract('doy' from g.game_dttm)  - extract('doy'from app.application_dttm)

	end) as days_between_app_gamedt

from

	msu_analytics.client c

	left join msu_analytics.account acc on c.client_rk = acc.client_rk

	left join msu_analytics.application app on acc.account_rk = app.account_rk

	left join msu_analytics.game g on app.game_rk = g.game_rk

order by c.visit_dttm

```

